# Time series comparison between FVCOM and MPOS
**Author: Jun Sasaki, Coded on January 7, 2025, Updated on January 8, 2025 ([MIT License](https://opensource.org/licenses/MIT))**<br>
Compare FVCOM output netcdf and Mpos data at a specified node/nele and siglay.
- Suppose Mpos netCDF and FVCOM output netCDF are prepared.
- FVCOM variables are defined at siglay. The depth at a specified siglay will change with time because of the change in the surface level.
- Interpolate Mpos item values at the specified siglay in FVCOM to precisely compare them.

## Methods
1. Load the FVCOM output netcdf file into xfvcom, specify the node or nele value for the horizontal coordinate in the scalar and the siglay or siglev values for the vertical coordinate in the list, extract the corresponding parts, and plot them.
2. Load the Mpos observation data and extract the data closest to the datetime and depth corresponding to 1. In doing so, set the allowable error and extract the data.
3. Extract the common parts of 1 and 2.
4. Use 3 to evaluate the accuracy of the calculation results.

In [ ]:
from xfvcom import FvcomDataLoader, FvcomAnalyzer, FvcomPlotConfig, FvcomPlotter
from xmpos import MposDataLoader, MposAnalyzer, MposPlotConfig, MposPlotter
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import hvplot.xarray
import panel.widgets as pnw
import holoviews as hv
## Global parameters
width=800; height=200

### Load FVCOM output netcdf into xarray.Dataset

In [ ]:
base_path = "~/Github/TB-FVCOM/goto2023/output"
ncfiles = ["TokyoBay18_r16_crossed_0001.nc"]
index_ncfile=0
fvcom = FvcomDataLoader(base_path=base_path, ncfile=ncfiles[index_ncfile], time_tolerance=5)
ds_fvcom = fvcom.ds
ds_fvcom

### Select a specified `node` of MPOS
Note: `nele` is unsupported.

In [ ]:
bl_stns = [(140.023333, 35.610833),  # 01kemigawa
           (139.833889, 35.490278),  # 02kawasaki
           (139.941667, 35.640000),  # 03urayasu
           (139.954167, 35.536944)]  # 04chiba1gou
stns = ["01kemigawa", "02kawasaki", "03urayasu", "04chiba1gou"]
index_stn = 0
stn = stns[index_stn]
lon, lat = bl_stns[index_stn]
f_a = FvcomAnalyzer(ds_fvcom)
node = f_a.nearest_neighbor(lon, lat, node=True, distances=False)
print(f"node={node} for ({lon}, {lat})")

### Slice `ds_fvcom` with specifying `var_name` and siglay and plot its time-series.
First, quick plot interactively.

In [ ]:
var_name = 'temp'
siglay_slider =  pnw.IntSlider(name='siglay', start=0, end=len(ds_fvcom.siglay) - 1, value=0)
ds_fvcom[var_name].isel(node=node).interactive.isel(siglay=siglay_slider).hvplot().opts(width=width, height=height)

Multiple siglays can be plotted by defining a list of `siglays` and `colors`.

In [ ]:
start=None; end=None
siglays=[10, 20, 29]  # siglay values
for siglay in siglays:
    print(f"siglay={siglay}: {ds_fvcom.z_dfs[:,siglay,node].min().item()} <= z <= {ds_fvcom.z_dfs[:,siglay,node].max().item()} ")
colors = ['blue', 'black', 'red']  # Line colors
plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(ds_fvcom, plot_config)
save_path = f"fc_ts_{var_name}.png"
fig, ax = plt.subplots(1, 1, figsize=(8, 2)) 
for i, siglay in enumerate(siglays):
    if i < len(siglays) - 1:
        ax=plotter.plot_timeseries(var_name, log=False, index=node, k=siglay, start=start, end=end, ax=ax, color=colors[i], alpha=0.7)
    else:
        plotter.plot_timeseries(var_name, log=False, index=node, k=siglay, start=start, end=end, ax=ax, color=colors[i], alpha=0.7, save_path=save_path)

### Extract time-series depth from the surface, `z_dfs`, at the specified `node` and `siglay`.

In [ ]:
fvcom_z_dfs=[]
for i, siglay in enumerate(siglays):
    fvcom_z_dfs.append(ds_fvcom.z_dfs[:,siglay,node])
    
combined_plot = fvcom_z_dfs[0].hvplot(label=f"siglay={siglays[0]}")
if len(fvcom_z_dfs) >1:
    for i, da in enumerate(fvcom_z_dfs[1:]):
        combined_plot *= da.hvplot(label=f"siglay={siglays[i+1]}")
combined_plot.opts(width=width, height=height, legend_position='right', invert_yaxis=True)

## Load vertically structured Mpos.

In [ ]:
base_path = '~/Github/xmpos/data/MLIT_edited_Mpos/nc_structured/'
# base_path = '~/Github/TB-FVCOM/data/MLIT_edited_Mpos/nc/'
# kemigawa = loader.load_nc(stn='01kemigawa', start=2010, end=2020, dask=False)
loader = MposDataLoader(base_path=base_path)
ds_mpos = loader.load_structured_nc(stn=stn, start=2015, end=2020, dask=False)
#print(ds_mpos.temp[30000:,5].values)
#print(ds_mpos.z[5].item())

### Plot time-series of vertical profile

In [ ]:
time_slice=(pd.Timestamp('2020-01-01'), pd.Timestamp('2020-01-15'))
mpos_config = MposPlotConfig(figsize=(8, 2), dpi=72, cmap="jet", levels=21)
plotter = MposPlotter(ds_mpos, config=mpos_config)
# Create the contour plot
fig, ax, cbar = plotter.plot_contour(
    varname=var_name, xlim=('2015-01-01', '2021-02-01'), ylim=(0, 9), clim=(10, 13),
    stn=stn, rolling=False, window=25
)
# Adjust the x-axis and add more annotations
#ax.set_xlim(pd.Timestamp('2020-01-01'), pd.Timestamp('2020-01-15'))
ax.set_xlim(time_slice)
#ax.set_title("Adjusted DO at Kemigawa")
plt.show()

### Time-series interactive plot at each z 

In [ ]:
z_slider =  pnw.IntSlider(name='z', start=0, end=len(ds_mpos.z) - 1, value=0)
ds_mpos[var_name].sel(time=slice(time_slice[0], time_slice[1])).interactive.isel(z=z_slider).hvplot().opts(width=700, height=height)

### Interpolate ds_mpos at time-series depth of FVCOM `z_dfs` with interactive plot
`da_mpos_interp_list` is a list of DataArrays.

In [ ]:
hv.renderer('bokeh').theme = 'caliber'
width=800; height=200
size=20; alpha=0.5; marker='o'

mpos_analyzer = MposAnalyzer(ds_mpos)

da_mpos_interp_list = []
for i, siglay in enumerate(siglays):
    da_mpos_interp_list.append(mpos_analyzer.interpolate_timeseries_by_depth(var_name, fvcom_z_dfs[i]))

combined_plot = da_mpos_interp_list[0].hvplot.line(alpha=alpha, label=f"siglay={siglays[0]}") \
              * da_mpos_interp_list[0].hvplot.scatter(marker=marker, size=size, alpha=alpha)
if len(da_mpos_interp_list) >1:
    for i, da in enumerate(da_mpos_interp_list[1:]):
        combined_plot *= da.hvplot.line(alpha=alpha, label=f"siglay={siglays[i+1]}")
        combined_plot *= da.hvplot.scatter(marker=marker, size=size, alpha=alpha)
combined_plot.opts(width=width, height=height, legend_position='right')
hvplot.save(combined_plot.options(toolbar=None), f"MPOS {var_name} at siglays={siglays}.html")
combined_plot

### Compare ds_fvcom[var_name] and MPOS
1. `da_mpos_interp_list` is a list of MPOS DataArrays. `da_fvcom = ds_fvcom[var_name].isel(node=node, siglay=siglay))` is a DataArray of FVCOM. Its list is defined as `da_fvcom_list`.
2. When either of the two DataArrays contains missing values, a `common_mask` is created to mark those time values as missing.
3. Apply `common_mask` to both DataArrays and create DataArray lists of `da_fvcom_cleaned_list` and `da_mpos_interp_cleaned_list`.

In [ ]:
da_fvcom_list = []
for i in range(len(siglays)):
    da = ds_fvcom[var_name].isel(node=node, siglay=siglays[i])
    da_fvcom_list.append(da)
common_mask = []
da_mpos_interp_cleaned_list = []
da_fvcom_cleaned_list = []
for i in range(len(siglays)):
    common_mask.append(~np.isnan(da_mpos_interp_list[i]) & ~np.isnan(da_fvcom_list[i]))
    da_mpos_interp_cleaned_list.append(da_mpos_interp_list[i].where(common_mask[i]))
    da_fvcom_cleaned_list.append(da_fvcom_list[i].where(common_mask[i]))

combined_plot = da_mpos_interp_cleaned_list[0].hvplot.line(alpha=alpha, label=f"siglay={siglays[0]}") \
              * da_fvcom_cleaned_list[0].hvplot.scatter(marker=marker, size=size, alpha=alpha)

if len(siglays) >1:
    for i in range(1, len(siglays)):
        combined_plot *= da_mpos_interp_cleaned_list[i].hvplot.line(alpha=alpha, label=f"siglay={siglays[i]}")
        combined_plot *= da_fvcom_cleaned_list[i].hvplot.scatter(marker=marker, size=size, alpha=alpha)

combined_plot.opts(width=width, height=height, legend_position='right')
hvplot.save(combined_plot.options(toolbar=None), f"FVCOM-MPOS {var_name} at siglays={siglays}.html")
combined_plot
